# Wall project

In [16]:
import numpy as np
import libMobility as lm
import logging
import os
from tqdm import tqdm
logging.basicConfig(level=logging.DEBUG)
precision = np.float32 if lm.SelfMobility.precision == "float" else np.float64


def create_square_patch(side, density):
    ''' Create a square patch of particles in the XY plane, centered at 0,0 with a given density.'''
    # Density is the number of particles per unit area in the square patch
    n = int(side * side * density)  # Number of particles in the square patch
    nx = int(np.sqrt(n))            # Number of particles in the x direction
    ny = nx                         # Number of particles in the y direction (equal to x)
    n = nx * ny                     # Real total number of particles
    sep = side / nx                 # Separation between particles
    pos = np.zeros((n, 3))          # Initialize positions array
    # Set positions in the XY plane
    pos = np.array(                 
        [
            [i * sep - side * 0.5, j * sep - side * 0.5, 0]
            for i in range(nx)
            for j in range(ny)
        ]
    )
    return pos


def create_tracer_box(lxy, lz, density):
    """Create a box centered at 0,0 in XY and such that max(z) = lz"""
    # Density is the number of particles per unit volume in the rectangular box
    nxy = int(np.sqrt(np.cbrt(density) ** 2 * lxy * lxy)) # Number of particles in the XY plane
    nz = int(np.cbrt(density) * lz)                 # Number of particles in the Z direction
    x = np.arange(-lxy * 0.5, lxy * 0.5, lxy / nxy) # X positions
    y = x.copy()                                    # Y positions (same as X)   
    z = np.arange(0, lz, lz / nz)                   # Z positions
    xx, yy, zz = np.meshgrid(x, y, z, indexing="ij")    # Create a meshgrid for X, Y, Z positions
    pos = np.column_stack((xx.ravel(), yy.ravel(), zz.ravel())) # Stack the positions into a single array
    return pos


def init_solver(numberParticles):
    ''' Initialize the solver with the given number of particles.'''
    nb = lm.NBody(periodicityX="open", periodicityY="open", periodicityZ="open")    # Boundary conditions
    nb.setParameters(algorithm="advise", Nbatch=1, NperBatch=numberParticles)       # Set parameters for the solver
    # Initialize physical parameters
    nb.initialize(
        temperature=0.0,
        viscosity=1 / (6 * np.pi),
        hydrodynamicRadius=1.0,
        numberParticles=numberParticles,
        needsTorque=False,
    )
    return nb


class FixedPointSprings:
    '''Class to compute the forces on particles due to fixed point springs.'''
    def __init__(self, kspring, indexes, pos):
        self.kspring = kspring          # Spring constant
        self.indexes = indexes.copy()   # Indexes of the particles connected by springs
        self.pos = pos.copy()           # Equilibrium positions of the particles connected by springs

    # Force calculation method
    def compute(self, all_pos): 
        delta = all_pos[self.indexes] - self.pos    # Calculate the displacement from equilibrium
        return -self.kspring * delta.copy()         # Calculate the spring force

# Structural parameters
quadrupole_width = 20                # Width of the quadrupole patch
quadrupole_density = 2              # Density of the quadrupole patch
tracer_width = quadrupole_width     # Width of the tracer box
tracer_density = 1                  # Density of the tracer box
kspring = 10                        # Spring constant
dt = 0.001                          # Time step for the simulation
pull = 10                           # Pulling force applied to the fixed particle
depth = 10                           # Depth of the tracer box
pull_radius = 5                     # Radius of the pulling particle

# System structure
wall_pos = create_square_patch(quadrupole_width, quadrupole_density) + np.array([0, 0, depth])             
tracers = create_tracer_box(tracer_width, depth, tracer_density)    # Create a box of tracers
pull_particle = np.array([0, 0, depth + 1], dtype=precision)        # Pulling particle position
all_pos = np.vstack((wall_pos, tracers, pull_particle))             # Combine all positions

radii = np.ones(all_pos.shape[0], dtype=precision) * 0.5        # Radii of all particles
radii[-1] *= pull_radius                                        # Set the radius of the pulling particle

pull_force = np.array([0, 0, pull], dtype=precision)                # Pulling force vector
f_tracers = np.zeros_like(tracers, dtype=precision)                 # Initialize forces on tracers
springs = FixedPointSprings(kspring, np.arange(wall_pos.shape[0]), wall_pos) # Create springs for the wall particles

all_forces = np.vstack((np.zeros_like(wall_pos), f_tracers, pull_force))     # Combine forces on all particles

solver = init_solver(all_pos.shape[0])                          # Initialize the solver

logging.info(f"Total particles: {all_pos.shape[0]}")

# Initialize a writing file
if os.path.exists("traj.txt"):
    os.remove("traj.txt")
else:
    logging.info("The file does not exist, it will be created.")


# Write the initial positions to the file
with open("traj.txt", "w") as f:
    for j in range(all_pos.shape[0]):
        f.write(f"{all_pos[j, 0]} {all_pos[j, 1]} {all_pos[j, 2]} {radii[j]}\n")
    f.write("\n")

for i in tqdm(range(100)):
    solver.setPositions(all_pos)
    all_forces = np.zeros_like(all_pos)
    all_forces[springs.indexes] = springs.compute(all_pos)
    all_forces[-1] = pull_force
    result, _ = solver.Mdot(forces=all_forces)
    result[-1] = 0
    all_pos = all_pos + result * dt
    # Print, for each line: X Y Z 0.25, v. Leave a newline space for each frame
    with open(f"traj.txt", "a") as f:  # Use "a" mode to append instead of overwriting
        for j in range(all_pos.shape[0]):
            f.write(f"{all_pos[j, 0]} {all_pos[j, 1]} {all_pos[j, 2]} {radii[j]}\n")
        f.write("\n")


INFO:root:Total particles: 4785
100%|██████████| 100/100 [00:01<00:00, 60.42it/s]
